<a href="https://colab.research.google.com/github/JSJeong-me/GPT-Web/blob/main/213-Streaming-Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

위의 챗봇의 사용자 경험을 개선할 수 있는 방법에는 여러 가지가 있습니다. 첫째, 사용자가 메시지가 생성될 때까지 오래 기다릴 필요가 없도록 응답을 스트리밍할 수 있습니다. 둘째, 챗봇의 응답이 생성되는 동안 사용자 메시지가 채팅 기록에 즉시 표시되도록 할 수 있습니다. 이를 달성하기 위한 코드는 다음과 같습니다.

사용자가 메시지를 제출하면 이제 .then()을 사용하여 세 가지 이벤트 이벤트를 연결하는 것을 알 수 있습니다.

첫 번째 메소드 user()는 사용자 메시지로 챗봇을 업데이트하고 입력 필드를 지웁니다. 이 방법은 또한 입력 필드를 비대화형으로 만들어 챗봇이 응답하는 동안 사용자가 다른 메시지를 보낼 수 없도록 합니다. 이 작업이 즉시 수행되기를 원하기 때문에 queue=False로 설정하여 활성화된 대기열을 건너뜁니다. 챗봇의 기록에는 (user_message, None)이 추가됩니다. None은 봇이 응답하지 않았음을 나타냅니다.

두 번째 메소드인 bot()은 봇의 응답으로 챗봇 기록을 업데이트합니다. 새 메시지를 생성하는 대신 이전에 생성된 None 메시지를 봇의 응답으로 대체합니다. 마지막으로 메시지를 문자별로 구성하고 구성되는 대로 중간 출력을 생성합니다. Gradio는 Yield 키워드가 포함된 모든 기능을 스트리밍 출력 인터페이스로 자동으로 전환합니다.

세 번째 방법은 사용자가 봇에 다른 메시지를 보낼 수 있도록 입력 필드를 다시 대화형으로 만듭니다.

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr
import random
import time

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    def user(user_message, history):
        return "", history + [[user_message, None]]

    def bot(history):
        bot_message = random.choice(["How are you?", "I love you", "I'm very hungry"])
        history[-1][1] = ""
        for character in bot_message:
            history[-1][1] += character
            time.sleep(0.05)
            yield history

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    clear.click(lambda: None, None, chatbot, queue=False)

demo.queue()
demo.launch()
